### Mini E-commerce Order Processor

Build a tiny order processing system that follows **all five SOLID principles.**  

**Requirements**:
- Order with items and total calculation
- At least two payment methods (CC, PayPal)
- At least two notification methods (Email, SMS)
- Persistence (save order, print or write to JSON)
- Must be extensible; adding a new payment or notification method requires zero changes to existing classes

In [2]:
# Mini E-Commerce Order Processor

import json
from abc import ABC, abstractmethod
from typing import List
from dataclasses import dataclass, field
from enum import Enum
from datetime import datetime
from random import randint
from pathlib import Path
import smtplib
from email.mime.text import MIMEText
import requests

# --- SOLID PRINCIPLES IMPLEMENTATION ---

# Single Responsibility Principle (SRP)
# Explanation: Item and Order classes have a single responsibility:
# Item represents a product, and Order manages a collection of items.
@dataclass
class Item:
    name: str
    price: float
    quantity: int = 1

@dataclass
class Order:
    items: List[Item] = field(default_factory=list)

    def add_item(self, item: Item):
        self.items.append(item)

    def total(self) -> float:
        return sum(item.price * item.quantity for item in self.items)

# Open/Closed Principle (OCP)
# Explanation: PaymentMethod is open for extension (new payment methods can be added)
# but closed for modification (existing code does not need to be changed).
class PaymentMethod(ABC):
    @abstractmethod
    def pay(self, amount: float) -> bool:
        pass

class CreditCardPayment(PaymentMethod):
    def __init__(self, card_number: str, cvv: str, expiry_date: str):
        self.card_number = card_number
        self.cvv = cvv
        self.expiry_date = expiry_date

    def pay(self, amount: float) -> bool:
        print(f"Processing credit card payment of ${amount:.2f}")
        return True

class PayPalPayment(PaymentMethod):
    def __init__(self, email: str, password: str):
        self.email = email
        self.password = password

    def pay(self, amount: float) -> bool:
        print(f"Processing PayPal payment of ${amount:.2f}")
        return True

# Liskov Substitution Principle (LSP)
# Explanation: Both CreditCardPayment and PayPalPayment can be used interchangeably
# wherever PaymentMethod is expected without altering the correctness of the program.
class NotificationMethod(ABC):
    @abstractmethod
    def notify(self, message: str):
        pass

class EmailNotification(NotificationMethod):
    def __init__(self, email_address: str):
        self.email_address = email_address

    def notify(self, message: str):
        print(f"Sending email to {self.email_address}: {message}")
        # Here you would implement actual email sending logic

class SMSNotification(NotificationMethod):
    def __init__(self, phone_number: str):
        self.phone_number = phone_number

    def notify(self, message: str):
        print(f"Sending SMS to {self.phone_number}: {message}")
        # Here you would implement actual SMS sending logic

# Interface Segregation Principle (ISP)
# Explanation: OrderPersistence interface is separate from PaymentMethod and NotificationMethod,
# allowing classes to implement only the interfaces they need.
class OrderPersistence(ABC):
    @abstractmethod
    def save(self, order: Order):
        pass

class JSONOrderPersistence(OrderPersistence):
    def __init__(self, filepath: str):
        self.filepath = filepath

    def save(self, order: Order):
        order_data = {
            "items": [{"name": item.name, "price": item.price, "quantity": item.quantity} for item in order.items],
            "total": order.total()
        }
        with open(self.filepath, 'w') as f:
            json.dump(order_data, f, indent=4)
        print(f"Order saved to {self.filepath}")

# Dependency Inversion Principle (DIP)
# Explanation: OrderProcessor depends on abstractions (PaymentMethod, NotificationMethod, OrderPersistence)
# rather than concrete implementations.
class OrderProcessor:
    def __init__(self, payment_method: PaymentMethod, notification_method: NotificationMethod, persistence: OrderPersistence):
        self.payment_method = payment_method
        self.notification_method = notification_method
        self.persistence = persistence

    def process_order(self, order: Order):
        amount = order.total()
        if self.payment_method.pay(amount):
            self.persistence.save(order)
            self.notification_method.notify(f"Your order of ${amount:.2f} has been processed successfully.")
        else:
            self.notification_method.notify("Payment failed. Please try again.")

# Example Usage
if __name__ == "__main__":
    # Create an order
    order = Order()
    order.add_item(Item(name="GeForce RTX 3050", price=205.00, quantity=2))
    order.add_item(Item(name="RTX 3090", price=1700.00, quantity=1))
    order.add_item(Item(name="NVIDIA DGX Spark", price=4000.00, quantity=1))

    # Choose payment and notification methods
    payment_method = CreditCardPayment(card_number="1234567890123456", cvv="123", expiry_date="12/29")
    notification_method = EmailNotification(email_address="customer@commons.com")
    persistence = JSONOrderPersistence(filepath="order.json")

    # Process the order
    processor = OrderProcessor(payment_method, notification_method, persistence)
    processor.process_order(order)



Processing credit card payment of $6110.00
Order saved to order.json
Sending email to customer@commons.com: Your order of $6110.00 has been processed successfully.


### Explanation

**(SRP) Single Responsibility Principle**:  
Each class has a single responsibility. The Item class represents a product with its details, and the Order class manages a collection of items and calculates the total cost.  

**(OCP) Open/Closed Principle**:  
The PaymentMethod class is open for extension (we can add new payment methods)
but closed for modification (we don't need to change existing code to add new methods).  

**(LSP) Liskov Substitution Principle**:  
Both CreditCardPayment and PayPalPayment can be used interchangeably wherever PaymentMethod is expected without altering the correctness of the program.  

**(ISP) Interface Segregation Principle**:  
OrderPersistence interface is separate from PaymentMethod and NotificationMethod, allowing classes to implement only the interfaces they need.  

**(DIP) Dependency Inversion Principle**:  
OrderProcessor depends on abstractions (PaymentMethod, NotificationMethod, OrderPersistence) rather than concrete implementations.